In [ ]:
# Before running this script as it currently stands, you'll need to run two notebooks:
# 1. ejscreen_etl.ipynb
# 2. score_calc_0.1.ipynb

import numpy as np
import pandas as pd
from pathlib import Path
import requests
import zipfile

In [ ]:
# Suppress scientific notation in pandas (this shows up for census tract IDs)
pd.options.display.float_format = "{:.2f}".format

# Set some global parameters
DATA_DIR = Path.cwd().parent / "data"
TEMP_DATA_DIR = Path.cwd().parent / "data" / "tmp"
# None of these numbers are final, but just for the purposes of comparison.
CALENVIROSCREEN_PRIORITY_COMMUNITY_THRESHOLD = 75
CEJST_PRIORITY_COMMUNITY_THRESHOLD = 0.75

# Name fields using variables. (This makes it easy to reference the same fields frequently without using strings
# and introducing the risk of misspelling the field name.)
CENSUS_BLOCK_GROUP_ID_FIELD = "census_block_group_id"
CENSUS_BLOCK_GROUP_POPULATION_FIELD = "census_block_group_population"
CENSUS_TRACT_ID_FIELD = "census_tract_id"
CALENVIROSCREEN_SCORE_FIELD = "calenviroscreen_score"
CALENVIROSCREEN_PERCENTILE_FIELD = "calenviroscreen_percentile"
CALENVIROSCREEN_PRIORITY_COMMUNITY_FIELD = "calenviroscreen_priority_community"

# Note: we are pretending the EJSCREEN's low income percent is the actual score for now as a placeholder.
CEJST_SCORE_FIELD = "cejst_score"
CEJST_PERCENTILE_FIELD = "cejst_percentile"
CEJST_PRIORITY_COMMUNITY_FIELD = "cejst_priority_community"

# Comparison field names
tract_has_at_least_one_cbg = "CES Tract has at least one CEJST CBG?"
tract_has_100_percent_cbg = "CES Tract has 100% CEJST CBGs?"

In [ ]:
# Load CEJST score data
cejst_data_path = DATA_DIR / "score" / "csv" / "usa.csv"

cejst_df = pd.read_csv(cejst_data_path)

cejst_df.head()

# Rename unclear name "id" to "census_block_group_id", as well as other renamings.
cejst_df.rename(
    columns={
        "ID": CENSUS_BLOCK_GROUP_ID_FIELD,
        "ACSTOTPOP": CENSUS_BLOCK_GROUP_POPULATION_FIELD,
        "score_a": CEJST_SCORE_FIELD,
        "score_a_percentile": CEJST_PERCENTILE_FIELD,
    },
    inplace=True,
    errors="raise",
)

# Calculate the top K% of prioritized communities
cejst_df[CEJST_PRIORITY_COMMUNITY_FIELD] = (
    cejst_df[CEJST_PERCENTILE_FIELD] >= CEJST_PRIORITY_COMMUNITY_THRESHOLD
)

# Create the CBG's Census Tract ID by dropping the last number from the FIPS CODE of the CBG.
# The CBG ID is the last one character.
# For more information, see https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html.
cejst_df.loc[:, CENSUS_TRACT_ID_FIELD] = (
    cejst_df.loc[:, CENSUS_BLOCK_GROUP_ID_FIELD].astype(str).str[:-1].astype(np.int64)
)

# Remove all non-California data
cejst_df = cejst_df.loc[
    cejst_df[CENSUS_BLOCK_GROUP_ID_FIELD].astype(str).str[0] == "6", :
]

cejst_df.head()

In [ ]:
# Load comparison index (CalEnviroScreen 4)
# TODO: Instead of loading a manually-created CSV from the repo, actually download the data and convert it using
# Python ETL scripts.
# See ticket https://github.com/usds/justice40-tool/issues/194. 
# Data from https://calenviroscreen-oehha.hub.arcgis.com/#Data, specifically:
# https://oehha.ca.gov/media/downloads/calenviroscreen/document/calenviroscreen40resultsdatadictionaryd12021.zip
calenviroscreen_4_csv_name = "CalEnviroScreen_4.0_2021.csv"
calenviroscreen_data_path = DATA_DIR.joinpath(calenviroscreen_4_csv_name)

calenviroscreen_df = pd.read_csv(calenviroscreen_data_path)

calenviroscreen_df.rename(
    columns={
        "Census Tract": CENSUS_TRACT_ID_FIELD,
        "DRAFT CES 4.0 Score": CALENVIROSCREEN_SCORE_FIELD,
        "DRAFT CES 4.0 Percentile": CALENVIROSCREEN_PERCENTILE_FIELD,
    },
    inplace=True,
)


# Calculate the top K% of prioritized communities
calenviroscreen_df[CALENVIROSCREEN_PRIORITY_COMMUNITY_FIELD] = (
    calenviroscreen_df[CALENVIROSCREEN_PERCENTILE_FIELD]
    >= CALENVIROSCREEN_PRIORITY_COMMUNITY_THRESHOLD
)

calenviroscreen_df.head()

In [ ]:
# Join CalEnviroScreen and CEJST data.
# Note: we're joining on the census *tract*, so there will be multiple CBG entries joined to the same census tract row from CES,
# creating multiple rows of the same CES data.

# For simplicity, we'll only keep certain columns from each data frame.
cejst_columns_to_keep = [
    CENSUS_BLOCK_GROUP_ID_FIELD,
    CENSUS_TRACT_ID_FIELD,
    CENSUS_BLOCK_GROUP_POPULATION_FIELD,
    CEJST_SCORE_FIELD,
    CEJST_PERCENTILE_FIELD,
    CEJST_PRIORITY_COMMUNITY_FIELD,
]

calenviroscreen_columns_to_keep = [
    CENSUS_TRACT_ID_FIELD,
    CALENVIROSCREEN_SCORE_FIELD,
    CALENVIROSCREEN_PERCENTILE_FIELD,
    CALENVIROSCREEN_PRIORITY_COMMUNITY_FIELD,
]

merged_df = cejst_df.loc[:, cejst_columns_to_keep].merge(
    calenviroscreen_df.loc[:, calenviroscreen_columns_to_keep],
    how="left",
    on=CENSUS_TRACT_ID_FIELD,
)

merged_df.head()

# merged_df.to_csv(
#     path_or_buf=TEMP_DATA_DIR / "merged.csv",
#     na_rep="",
#     index=False
# )

In [ ]:
# Create analysis
def calculate_comparison(frame):
    # Keep all the CES values at the Census Tract Level
    df = frame.loc[
        frame.index[0],
        [
            CENSUS_TRACT_ID_FIELD,
            CALENVIROSCREEN_SCORE_FIELD,
            CALENVIROSCREEN_PERCENTILE_FIELD,
            CALENVIROSCREEN_PRIORITY_COMMUNITY_FIELD,
        ],
    ]

    # Convenience constant for whether the tract is or is not a CalEnviroScreen priority community.
    is_a_ces_priority_tract = frame.loc[
        frame.index[0], [CALENVIROSCREEN_PRIORITY_COMMUNITY_FIELD]
    ][0]

    # Recall that NaN values are not falsy, so we need to check if `is_a_ces_priority_tract` is True.
    is_a_ces_priority_tract = is_a_ces_priority_tract is True

    # Calculate comparison
    df[tract_has_at_least_one_cbg] = (
        frame.loc[:, CEJST_PRIORITY_COMMUNITY_FIELD].sum() > 0
        if is_a_ces_priority_tract
        else None
    )
    df[tract_has_100_percent_cbg] = (
        frame.loc[:, CEJST_PRIORITY_COMMUNITY_FIELD].mean() == 1
        if is_a_ces_priority_tract
        else None
    )

    return df


# Group all data by the census tract.
grouped_df = merged_df.groupby(CENSUS_TRACT_ID_FIELD)

# Run the comparison function on the groups.
comparison_df = grouped_df.apply(calculate_comparison)

# Sort descending by highest CES Score for convenience when viewing output file
comparison_df.sort_values(
    by=[CALENVIROSCREEN_PERCENTILE_FIELD], ascending=False, inplace=True
)

# Write comparison to CSV.
comparison_df.to_csv(
    path_or_buf=TEMP_DATA_DIR / "Comparison Output.csv", na_rep="", index=False
)

print(comparison_df.head())

In [ ]:
# Prepare some constants for use in the following Markdown cell.

cejst_cbgs_ca_only = cejst_df.loc[:, CEJST_PRIORITY_COMMUNITY_FIELD].sum()
ces_tracts_count = comparison_df.loc[:, CALENVIROSCREEN_PRIORITY_COMMUNITY_FIELD].sum()
at_least_one_sum = comparison_df.loc[:, tract_has_at_least_one_cbg].sum()
at_least_one_sum_percent = f"{at_least_one_sum / ces_tracts_count:.0%}"

all_100_sum = comparison_df.loc[:, tract_has_100_percent_cbg].sum()
all_100_sum_percent = f"{all_100_sum / ces_tracts_count:.0%}"

# Note, for the following Markdown cell to render the variables properly, follow the steps at
# "Activating variable-enabled Markdown for Jupyter notebooks" within `score/README.md`.

# Summary of findings

Recall that census tracts contain one or more census block groups, with up to nine census block groups per tract.

There are {{ces_tracts_count}} census tracts designated as Disadvantaged Communities by CalEnviroScreen 4.0. 

Within California, there are {{cejst_cbgs_ca_only}} census block groups considered as priority communities by the current version of the CEJST score used in this analysis.

Out of every CalEnviroScreen Disadvantaged Community census tract, {{at_least_one_sum}} ({{at_least_one_sum_percent}}) of these census tracts have at least one census block group within them that is considered a priority community by the current version of the CEJST score.

Out of every CalEnviroScreen Disadvantaged Community census tract, {{all_100_sum}} ({{all_100_sum_percent}}) of these census tracts have 100% of the included census block groups within them considered priority communities by the current version of the CEJST score.